# Initial sea ice diagnostics

In [2]:
from aqua import Reader,catalogue, inspect_catalogue
import datetime
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

FDB5 binary library not present on system, disabling FDB support.


In [3]:
#cat = catalogue()

# Parameters, to be moved to a YAML file eventually
                       # Region name          # southernmost latitude    # northernmost latitude   # westernmost longitude # easternmost longitude
regions = [ \
                       [ "Arctic",            [0.0,                      90.0,                     0.0,                    360.0                 ]], \
                       #[ "Southern Ocean",    [-90.0,                    0.0,                      0.0,                    360.0                 ]], \
                       #[ "Hudson Bay",        [50.0,                     63.0,                     265.0,                  285.0                 ]], \
                       #[ "Ross Sea",          [-90.0,                    0.0,                      160.0,                   230.0                ]], \
                       #[ "Amundsen-Bellingshausen Seas", [-90.0,         0.0,                      230.0,                  300.0                 ]], \
                       #[ "Weddell Sea",       [-90.0,                    0.0,                      300.0,                  20.0                  ]], \
                       #[ "Indian Ocean",      [-90.0,                    0.0,                      20.0,                   90.0                  ]], \
                       #[ "Pacific Ocean",     [-90.0,                    0.0,                      90.0,                   160.0                 ]], \
                    ]

thresholdSeaIceExtent = 0.15

In [4]:
# Exp specification
 
# Works
#model = "IFS"
#exp   = "tco1279-orca025-cycle3"
#source= "lra-r100-monthly"
#computeGridCellAreas  = True

# in test
#model = "FESOM"
#exp   = "tco2559-ng5-cycle3"
#source= "lra-r100-monthly"

model = "OSI-SAF"
exp   = "osi-450"
source = "nh"
dim1Name = "yc"
dim2Name = "xc"
fix   = True
computeGridCellAreas  = False


label = model + " " + exp + " " + source

In [10]:

# Instantiate reader
reader = Reader(model  = model, \
                exp    = exp,   \
                source = source,\
               )

data = reader.retrieve(fix = fix, regrid = "r100", startdate="2010-01-01", enddate="2015-12-31")


In [11]:
print(data.siconc)

<xarray.DataArray 'siconc' (time: 11801, yc: 432, xc: 432)>
dask.array<mul, shape=(11801, 432, 432), dtype=float64, chunksize=(366, 432, 432), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-01-02T12:00:00 ... 2015-12-31T12:00:00
    lon      (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat      (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc       (xc) float64 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
  * yc       (yc) float64 5.388e+03 5.362e+03 ... -5.362e+03 -5.388e+03
Attributes: (12/14)
    standard_name:        sea_ice_area_fraction
    long_name:            Sea ice area fraction
    units:                Fraction
    grid_mapping:         Lambert_Azimuthal_Grid
    ancillary_variables:  total_standard_error status_flag
    comment:              this field is the primary sea ice concentration est...
    ...                   ...
    cfVarName:            siconc
    shortName:            siconc


In [21]:

# Compute grid cell areas from regular grid if needed
if computeGridCellAreas:
    # Create grid cell areas
    # Ensure that the grid is really regular
    if len(data.lon.shape) != 1 or len(data.lat.shape) != 1 \
      or len(set(np.diff(data.lon))) != 1 or len(set(np.diff(data.lat))) != 1:
        print("Lat and lon are not regular")
        stop()
    else:

        # Broadcast lat and lon to 2-D arrays
        lon, lat = xr.broadcast(data.lon, data.lat)

        dlam = np.diff(data.lon)[0] * 2 * np.pi / 360.0 # Degrees to radians
        dphi = np.diff(data.lat)[0] * 2 * np.pi / 360.0 # Degrees to radians

        phi = 2 * np.pi / 360.0 * data.lat
        lam = 2 * np.pi / 360.0 * data.lon

        # Expand grid
        lam, phi = xr.broadcast(data.lon / 360.0 * 2 * np.pi, data.lat / 360.0 * 2 * np.pi)
 

        Rearth = 6356e3 # meters

        areacello = np.cos(phi) * Rearth * dlam * Rearth * dphi / 1e12
        areacello = areacello.rename("areacello")
        areacello.attrs["units"] = "million km^2"
        areacello.attrs["standard_name"] = "Grid cell area"

        # Check that sum is within the true Earth surface
        trueEarthSurface = 510072000e6 # Wikipedia
        if np.abs((areacello.sum() - trueEarthSurface) / trueEarthSurface) > 0.01:
            print("Earth's surface wrongly calculated")
else:
    areacello = reader.grid_area
    lat = data.lat
    lon = data.lon
                               

print(areacello.sum())

<xarray.DataArray 'cell_area' ()>
dask.array<sum-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] ...
Attributes:
    standard_name:      sea_ice_area_fraction status_flag
    long_name:          status flag bit array for sea ice concentration retri...
    grid_mapping:       Lambert_Azimuthal_Grid
    flag_masks:         [  1   2   4   8  16  32  64 128]
    flag_meanings:      land lake open_water_filtered land_spill_over high_t2...
    flag_descriptions:  \nall bits to 0 (flag 0): Nominal retrieval by the SI...
    comment:            Flag values found in the map might be combinations of...


In [23]:
# Mask based on threshold
siconc_mask = data.siconc.where(data.siconc > thresholdSeaIceExtent).where(data.siconc < 1.0)

print(siconc_mask.sum().data)

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>


In [27]:
# Iterate over regions
for jr, region in enumerate(regions):

    # Create regional mask
    latS, latN, lonW, lonE = regions[jr][1][0], regions[jr][1][1], regions[jr][1][2], regions[jr][1][3]

    # Dealing with regions straddling the 180° meridian
    if lonW > lonE:
        regionMask = (lat >= latS) & (lat <= latN) & ((lon >= lonW) | (lon <= lonE))
    else:
        regionMask = (lat >= latS) & (lat <= latN) & (lon >= lonW) & (lon <= lonE)

    areacello_mask = areacello.where(regionMask)

    
    print(lat.shape)
    

    # Create masks for summing later on
    areacello_mask = areacello.where(regionMask)

    extent = areacello.where(regionMask).where(siconc_mask.notnull()).sum(dim = [dim1Name, dim2Name])
    

    # Print region area for information
    #areaRegion = areacello.where(regionMask).sum()
    #print(region[0] + " region area: " + str(areaRegion.data) + areaRegion.units)
    fig, ax = plt.subplots(1, 1)
    extent.plot(label = label)
    ax.grid()
    ax.legend()
    ax.set_title("Sea ice extent: region " + region[0])
    regionNameNoSpace = "".join(region[0].split())
    fig.savefig("./fig_" + regionNameNoSpace + ".png")


(432, 432)


: 

: 